In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
import pickle
import datetime

In [3]:
df=pd.read_csv('F:\\Generative AI Course With Langchain and Huggingface\\13 - ANN Project Implementation\\annclassification\\Churn_Modelling.csv')
df.drop(["RowNumber","CustomerId","Surname"],axis=1,inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
lbe=LabelEncoder()
ohe=OneHotEncoder()
scalar=StandardScaler()

In [5]:
df['Gender']=lbe.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
geo=ohe.fit_transform(df[['Geography']])
eo=pd.DataFrame(geo.toarray(),columns=ohe.get_feature_names_out(["Geography"]))
input_data=pd.concat([df.drop("Geography",axis=1),eo],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [7]:
X=input_data.drop("EstimatedSalary",axis=1)
y=input_data["EstimatedSalary"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
X_train=scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)

In [8]:
with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)
with open('ohe.pkl','wb') as file:
    pickle.dump(ohe,file)
with open('lbe.pkl','wb') as file:
    pickle.dump(lbe,file)

In [9]:
# ANN-Regression

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [11]:
model=Sequential(
[
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) # o/p layer for regression default is Linear
])

In [12]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Setup dashboard
log_dir="log/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [14]:
# Setting uup early stopping
earlystopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [15]:
history=model.fit(
X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,earlystopping]
)

Epoch 1/100


235/235 [==============================] - 4s 8ms/step - loss: 100631.3125 - mae: 100631.3125 - val_loss: 98212.1562 - val_mae: 98212.1562
Epoch 2/100
235/235 [==============================] - 1s 5ms/step - loss: 100105.1641 - mae: 100105.1641 - val_loss: 97135.7188 - val_mae: 97135.7188
Epoch 3/100
235/235 [==============================] - 1s 5ms/step - loss: 98197.0547 - mae: 98197.0547 - val_loss: 94306.4375 - val_mae: 94306.4375
Epoch 4/100
235/235 [==============================] - 1s 5ms/step - loss: 94346.4141 - mae: 94346.4141 - val_loss: 89442.4453 - val_mae: 89442.4453
Epoch 5/100
235/235 [==============================] - 1s 5ms/step - loss: 88514.7812 - mae: 88514.7812 - val_loss: 82773.0391 - val_mae: 82773.0391
Epoch 6/100
235/235 [==============================] - 1s 6ms/step - loss: 81122.2266 - mae: 81122.2266 - val_loss: 75136.0547 - val_mae: 75136.0547
Epoch 7/100
235/235 [==============================] - 1s 5ms/step - loss: 73049.5078 - mae: 73049.5

In [16]:
# Load tensorboard
%load_ext tensorboard

In [19]:
%tensorboard --logdir log/20250211-125135

Reusing TensorBoard on port 6007 (pid 12776), started 0:00:18 ago. (Use '!kill 12776' to kill it.)

In [18]:
model.save('regressor.h5')

f:\Generative AI Course With Langchain and Huggingface\13 - ANN Project Implementation\ANN-Regression\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0
